In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gensim

In [3]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Flatten
# from tensorflow.keras.layers import Embedding
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import MaxPooling1D
# from tensorflow.keras import optimizers
# from keras.utils import np_utils
# from sklearn.preprocessing import LabelEncoder

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

from keras import optimizers

Using TensorFlow backend.


In [6]:
df = pd.read_pickle('./data/tokenized.pkl')
df['text_edit'] = df['text_edit'].apply(list)
df.head(5)

,text,sex,age,event,text_edit,token_len
0,57YOM WITH CONTUSION TO FACE AFTER STRIKING IT...,1,57,62,"[contus, face, strike, post, pounder, set, fen...",8
1,A 45YOM FELL ON ARM WHILE WORKING HAD SLIPPED ...,1,45,42,"[fell, arm, work, slip, water, fx, wrist]",7
2,58YOM WITH CERVICAL STRAIN BACK PAIN S P REST...,1,58,26,"[cervic, strain, back, pain, p, restrain, taxi...",16
3,33 YOM LAC TO HAND FROM A RAZOR KNIFE,1,33,60,"[lac, hand, razor, knife]",4
4,53YOM AT WORK IN A WAREHOUSE DOING UNSPECIFIED...,1,53,71,"[work, warehous, unspecifi, lift, strain, lo, ...",8


Create reference vocabulary used for training

In [7]:
# vocab = set()
# for _,e in df['text_edit'].iteritems():
#     vocab = vocab.union(set(e))
#
# len(vocab)

In [8]:
# len(vocab)

### Try Keras that fits the embedding

Determine maximum token sequence length for training. We don't want to simply use that max as reviews 20 or longer are rare. Based on distribution below we select a max length of 15.

In [9]:
df['token_len'].value_counts().sort_index()

1         3
2         4
3       465
4      2050
5      5694
6      9954
7     13875
8     16838
9     17009
10    16252
11    14736
12    13000
13    10920
14     9344
15     7513
16     5979
17     4335
18     2870
19     1641
20      835
21      369
22      179
23       66
24       14
25        8
26        2
28        1
Name: token_len, dtype: int64

In [10]:
max_len = 15

In [11]:
docs = list(df['text_edit'].str[:max_len])

In [12]:
tokenizer = Tokenizer()

In [13]:
tokenizer.fit_on_texts(docs)

In [14]:
encoded_doc = tokenizer.texts_to_sequences(docs)

In [15]:
encoded_doc[:3]

[[18, 45, 257, 453, 3850, 844, 382, 453],
 [7, 32, 1, 34, 165, 38, 28],
 [209, 10, 6, 3, 23, 154, 902, 117, 31, 765, 442, 355, 206, 90, 519]]

In [16]:
xtrain = pad_sequences(encoded_doc, maxlen=max_len, padding='post')

In [17]:
xtrain[0]

array([  18,   45,  257,  453, 3850,  844,  382,  453,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [18]:
vocab_size = len(tokenizer.word_index) + 1
# we add +1 to account for missing words

Create target matrix.  
As this is a multi-class model keras requires the target to be in the form of a matrix.

In [19]:
encoder = LabelEncoder()

In [20]:
encoder.fit(df['event'].values)

LabelEncoder()

In [21]:
yencoded = encoder.fit_transform(df['event'].values)

In [22]:
encoder.inverse_transform([42])

array([72])

In [23]:
ytrain = np_utils.to_categorical(yencoded)

In [24]:
target_size = len(np.unique(yencoded))
target_size

48

Define Model

In [32]:
# model = Sequential()
# model.add(Embedding(vocab_size, 1, input_length=max_len))
# model.add(Conv1D(filters=1, kernel_size=2, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(1, activation='relu'))
# model.add(Dense(target_size, activation='softmax'))

In [96]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(target_size, activation='softmax'))

In [97]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 15, 100)           2684500   
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 8, 32)             25632     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 4, 32)             0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 100)               12900     
_________________________________________________________________
dense_36 (Dense)             (None, 48)                4848      
Total params: 2,727,880
Trainable params: 2,727,880
Non-trainable params: 0
_________________________________________________________________


In [90]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001), metrics=['accuracy'])

In [87]:
model.fit(xtrain, ytrain, epochs=10, verbose=1, validation_split=.2, batch_size=32)

Train on 123164 samples, validate on 30792 samples
Epoch 1/10
123164/123164 [==============================] - 19s 150us/step - loss: 1.8503 - acc: 0.4781 - val_loss: 1.1337 - val_acc: 0.6716
Epoch 2/10
123164/123164 [==============================] - 18s 144us/step - loss: 0.9606 - acc: 0.7214 - val_loss: 0.8694 - val_acc: 0.7442
Epoch 3/10
123164/123164 [==============================] - 18s 145us/step - loss: 0.7785 - acc: 0.7678 - val_loss: 0.7802 - val_acc: 0.7658
Epoch 4/10
123164/123164 [==============================] - 18s 144us/step - loss: 0.6817 - acc: 0.7936 - val_loss: 0.7336 - val_acc: 0.7771
Epoch 5/10
123164/123164 [==============================] - 18s 145us/step - loss: 0.6157 - acc: 0.8117 - val_loss: 0.7085 - val_acc: 0.7856
Epoch 6/10
123164/123164 [==============================] - 18s 144us/step - loss: 0.5663 - acc: 0.8265 - val_loss: 0.6905 - val_acc: 0.7907
Epoch 7/10
123164/123164 [==============================] - 18s 144us/step - loss: 0.5264 - acc: 0.8387

In [91]:
model.fit(xtrain, ytrain, epochs=10, verbose=1, validation_split=.2, batch_size=16)

Train on 123164 samples, validate on 30792 samples
Epoch 1/10
123164/123164 [==============================] - 36s 293us/step - loss: 1.5824 - acc: 0.5475 - val_loss: 0.9696 - val_acc: 0.7208
Epoch 2/10
123164/123164 [==============================] - 35s 286us/step - loss: 0.8506 - acc: 0.7515 - val_loss: 0.8112 - val_acc: 0.7594
Epoch 3/10
123164/123164 [==============================] - 35s 286us/step - loss: 0.7100 - acc: 0.7863 - val_loss: 0.7352 - val_acc: 0.7788
Epoch 4/10
123164/123164 [==============================] - 35s 285us/step - loss: 0.6289 - acc: 0.8072 - val_loss: 0.7020 - val_acc: 0.7874
Epoch 5/10
123164/123164 [==============================] - 35s 285us/step - loss: 0.5711 - acc: 0.8245 - val_loss: 0.6877 - val_acc: 0.7893
Epoch 6/10
123164/123164 [==============================] - 35s 285us/step - loss: 0.5261 - acc: 0.8358 - val_loss: 0.6740 - val_acc: 0.7933
Epoch 7/10
123164/123164 [==============================] - 35s 286us/step - loss: 0.4873 - acc: 0.8478

In [98]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy'])

In [99]:
model.fit(xtrain, ytrain, epochs=10, verbose=1, validation_split=.2, batch_size=128)

Train on 123164 samples, validate on 30792 samples
Epoch 1/10
123164/123164 [==============================] - 5s 45us/step - loss: 1.1232 - acc: 0.6727 - val_loss: 0.7284 - val_acc: 0.7780
Epoch 2/10
123164/123164 [==============================] - 5s 37us/step - loss: 0.6022 - acc: 0.8120 - val_loss: 0.6580 - val_acc: 0.7960
Epoch 3/10
123164/123164 [==============================] - 5s 37us/step - loss: 0.4700 - acc: 0.8505 - val_loss: 0.6664 - val_acc: 0.7954
Epoch 4/10
123164/123164 [==============================] - 4s 36us/step - loss: 0.3794 - acc: 0.8785 - val_loss: 0.6934 - val_acc: 0.7962
Epoch 5/10
123164/123164 [==============================] - 5s 37us/step - loss: 0.3095 - acc: 0.9009 - val_loss: 0.7501 - val_acc: 0.7904
Epoch 6/10
123164/123164 [==============================] - 4s 36us/step - loss: 0.2528 - acc: 0.9184 - val_loss: 0.7925 - val_acc: 0.7875
Epoch 7/10
123164/123164 [==============================] - 5s 37us/step - loss: 0.2071 - acc: 0.9329 - val_loss: 0

In [38]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(lr=0.0005), metrics=['accuracy'])

In [39]:
model.fit(xtrain, ytrain, epochs=2,verbose=1,validation_split=.2)

Train on 123164 samples, validate on 30792 samples
Epoch 1/2
123164/123164 [==============================] - 17s 139us/step - loss: 0.5750 - acc: 0.8216 - val_loss: 0.7054 - val_acc: 0.7885
Epoch 2/2
123164/123164 [==============================] - 17s 136us/step - loss: 0.4716 - acc: 0.8504 - val_loss: 0.6966 - val_acc: 0.7923


In [40]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(lr=0.0001), metrics=['accuracy'])

In [41]:
model.fit(xtrain, ytrain, epochs=5,verbose=1,validation_split=.2, batch_size=256)

Train on 123164 samples, validate on 30792 samples
Epoch 1/5
123164/123164 [==============================] - 3s 21us/step - loss: 0.3410 - acc: 0.8946 - val_loss: 0.6832 - val_acc: 0.7992
Epoch 2/5
123164/123164 [==============================] - 2s 18us/step - loss: 0.3216 - acc: 0.9004 - val_loss: 0.6890 - val_acc: 0.7998
Epoch 3/5
123164/123164 [==============================] - 2s 18us/step - loss: 0.3089 - acc: 0.9047 - val_loss: 0.6975 - val_acc: 0.7984
Epoch 4/5
123164/123164 [==============================] - 2s 18us/step - loss: 0.2981 - acc: 0.9078 - val_loss: 0.7053 - val_acc: 0.7974
Epoch 5/5
123164/123164 [==============================] - 2s 18us/step - loss: 0.2888 - acc: 0.9108 - val_loss: 0.7138 - val_acc: 0.7960


In [78]:
ytrain.argmax(1)

(153956,)

In [85]:
yprob = model.predict(xtrain)

In [112]:
yprob[0].argmax()

33

In [86]:
ypred = yprob.argmax(1)

In [96]:
dfres = pd.DataFrame(data=np.column_stack([ytrain.argmax(1),ypred]),columns=['act','pred'])
dfres.head()

,act,pred
0,33,33
1,18,18
2,10,10
3,31,31
4,41,41


In [98]:
# pd.crosstab(dfres['act'],dfres['pred']).to_excel('./results/crosstab_accuracy.xlsx')

In [101]:
np.sum(dfres['act'] == dfres['pred'])/dfres.shape[0]

0.8799721998493076

In [107]:
pd.concat([dfres['act'].value_counts(),dfres['pred'].value_counts()],1).sort_index().fillna('')


,act,pred
0,2,
1,8935,9832
2,2236,1343
3,3256,3272
4,12,
5,36,2
6,50,
7,283,305
8,1016,956
9,97,52


### Fit on all data instead of using a validation set
Based on above fit we will fit 10 epochs

In [115]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(target_size, activation='softmax'))

In [116]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 15, 100)           2684500   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 8, 32)             25632     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 4, 32)             0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               12900     
_________________________________________________________________
dense_16 (Dense)             (None, 48)                4848      
Total params: 2,727,880
Trainable params: 2,727,880
Non-trainable params: 0
_________________________________________________________________


In [117]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(lr=0.0001), metrics=['accuracy'])

In [118]:
model.fit(xtrain, ytrain, epochs=10,verbose=1)

Epoch 1/10
153956/153956 [==============================] - 306s 2ms/step - loss: 1.6894 - acc: 0.5208
Epoch 2/10
153956/153956 [==============================] - 310s 2ms/step - loss: 0.9085 - acc: 0.7348
Epoch 3/10
153956/153956 [==============================] - 316s 2ms/step - loss: 0.7479 - acc: 0.7757
Epoch 4/10
153956/153956 [==============================] - 276s 2ms/step - loss: 0.6592 - acc: 0.7996
Epoch 5/10
153956/153956 [==============================] - 304s 2ms/step - loss: 0.5966 - acc: 0.8168
Epoch 6/10
153956/153956 [==============================] - 284s 2ms/step - loss: 0.5485 - acc: 0.8309
Epoch 7/10
153956/153956 [==============================] - 282s 2ms/step - loss: 0.5097 - acc: 0.8415
Epoch 8/10
153956/153956 [==============================] - 295s 2ms/step - loss: 0.4766 - acc: 0.8513
Epoch 9/10
153956/153956 [==============================] - 328s 2ms/step - loss: 0.4475 - acc: 0.8602
Epoch 10/10
153956/153956 [==============================] - 316s 2ms/ste

In [119]:
model.save('./models/keras_try1.h5')